In [1]:
import glob 
import os
import sys

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla
import random

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

world = client.get_world()
blueprint_library = world.get_blueprint_library()

In [3]:
# Set up the simulator in synchronous mode
settings = world.get_settings()
original_settings = settings
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.1
world.apply_settings(settings)

# Set up the TM in synchronous mode
traffic_manager = client.get_trafficmanager()
traffic_manager.set_synchronous_mode(True)

# Set a seed so behaviour can be repeated if necessary
traffic_manager.set_random_device_seed(0)
random.seed(0)

In [4]:
models = ['dodge', 'audi', 'model3', 'mini', 'mustang', 'lincoln', 'prius', 'nissan', 'crown', 'impala']
blueprints = []
for vehicle in blueprint_library.filter('*vehicle*'):
    if any(model in vehicle.id for model in models):
        blueprints.append(vehicle)

In [5]:
bp = random.choice(blueprints)

In [6]:
spawn_points = world.get_map().get_spawn_points()

In [7]:
n = 10
vehicles = []
for i, x in enumerate(random.sample(spawn_points, n)):
    temp = world.try_spawn_actor(random.choice(blueprints), x)
    if temp is not None:
        temp.set_autopilot(True)
        traffic_manager.ignore_lights_percentage(temp, random.randint(0,50))
        vehicles.append(temp)

In [9]:
ego_vehicle = random.choice(vehicles)

In [10]:
camera_init_trans = carla.Transform(carla.Location(x=-5, z=3), carla.Rotation(pitch=-20))
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

In [ ]:
import pygame
import numpy as np

class RenderObject(object):
    def __init__(self, width, height):
        init_image = np.random.randint(0,255,(height,width,3),dtype='uint8')
        self.surface = pygame.surfarray.make_surface(init_image.swapaxes(0,1))

# Camera sensor callback, reshapes raw data from camera into 2D RGB and applies to PyGame surface
def pygame_callback(data, obj):
    img = np.reshape(np.copy(data.raw_data), (data.height, data.width, 4))
    img = img[:,:,:3]
    img = img[:, :, ::-1]
    obj.surface = pygame.surfarray.make_surface(img.swapaxes(0,1))

In [ ]:
camera.listen(lambda image: pygame_callback(image, RenderObject))

In [ ]:
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

In [ ]:
renderObject = RenderObject(image_w, image_h)

In [ ]:
pygame.init()
gameDisplay = pygame.display.set_mode((image_w,image_h), pygame.HWSURFACE | pygame.DOUBLEBUF)
# Draw black to the display
gameDisplay.fill((0,0,0))
gameDisplay.blit(renderObject.surface, (0,0))
pygame.display.flip()

In [ ]:
crashed = False

while not crashed:
    # Advance the simulation time
    world.tick()
    # Update the display
    gameDisplay.blit(renderObject.surface, (0,0))
    pygame.display.flip()

    # Collect key press events
    for event in pygame.event.get():
        # If the window is closed, break the while loop
        if event.type == pygame.QUIT:
            pygame.quit()
            if (world and world.recording_enabled):
                client.stop_recorder()
            if world is not None:
                world.destroy()
            crashed = True

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_RETURN:
                ego_vehicle.set_autopilot(False)
            # TAB key switches vehicle
            if event.key == pygame.K_TAB:
                ego_vehicle.set_autopilot(True)
                ego_vehicle = random.choice(vehicles)
                # Ensure vehicle is still alive (might have been destroyed)
                if ego_vehicle.is_alive:
                    # Stop and remove the camera
                    camera.stop()
                    camera.destroy()

                    camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)
                    camera.listen(lambda image: pygame_callback(image, renderObject))

                    # Update PyGame window
                    gameDisplay.fill((0,0,0))               
                    gameDisplay.blit(renderObject.surface, (0,0))
                    pygame.display.flip()


In [12]:
for x in vehicles:
    print(x.id)

190
191
192
193
194
195
196
197
198
199


In [34]:
ego2 = world.get_actor(195)

In [23]:
print(ego2)

Actor(id=193, type=vehicle.dodge.charger_police)


In [24]:
print(ego_vehicle)

Actor(id=195, type=vehicle.audi.tt)


In [35]:
d = ego_vehicle.get_location().distance(ego2.get_location())
print(d)

0.0
